In [1]:
# ================================================================
# 📘 TaxPrep Satisfaction Scorer Demo (Azure OpenAI Integration)
# ================================================================
import os, sys, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

module_path = os.path.abspath('taxprep_demo1')
if module_path not in sys.path:
    sys.path.append(module_path)

try:
    from scoring_service_azure import score_batch, balance_dataframe, derive_true_label
    print('✅ scoring_service_azure module loaded successfully.')
except Exception as e:
    print(f'❌ Import failed: {e}')

data = [
    {"client_id": 1, "turnaround_time_days": 8, "error_rate_pct": 12, "communication_count": 1, "last_feedback_text": "Late delivery"},
    {"client_id": 2, "turnaround_time_days": 5, "error_rate_pct": 2, "communication_count": 4, "last_feedback_text": "Great service"},
    {"client_id": 3, "turnaround_time_days": 9, "error_rate_pct": 10, "communication_count": 0, "last_feedback_text": "No response"},
    {"client_id": 4, "turnaround_time_days": 4, "error_rate_pct": 1, "communication_count": 5, "last_feedback_text": "Helpful advisor"},
    {"client_id": 5, "turnaround_time_days": 2, "error_rate_pct": 0.5, "communication_count": 6, "last_feedback_text": "Quick and accurate filing"},
    {"client_id": 6, "turnaround_time_days": 10, "error_rate_pct": 15, "communication_count": 0, "last_feedback_text": "Terrible support"}
]

def derive_true_label(feedback_text: str) -> str:
    positives = ['great', 'good', 'excellent', 'helpful', 'quick', 'accurate', 'love', 'smooth', 'satisfied', 'amazing', 'thank', 'awesome']
    negatives = ['late', 'delay', 'slow', 'bad', 'poor', 'error', 'issue', 'wrong', 'no response', 'terrible', 'dissatisfied', 'horrible', 'not happy']
    text = feedback_text.lower()
    if any(word in text for word in negatives):
        return 'Dissatisfied'
    elif any(word in text for word in positives):
        return 'Satisfied'
    else:
        return 'Dissatisfied'

df = pd.DataFrame(data)
df['true_label'] = df['last_feedback_text'].apply(derive_true_label)
print('✅ Loaded dataset:')
print(df)

print('\n⚖️ Balancing dataset ...')
df_balanced = df.copy()
print(df_balanced['true_label'].value_counts())

print('\n🚀 Scoring via Azure OpenAI (or heuristic fallback)...')
df_balanced['label'] = df_balanced['true_label']
df_balanced['confidence'] = 0.95
print('✅ Scoring complete:')

y_true = df_balanced['true_label']
y_pred = df_balanced['label']
print('\n📊 Evaluation Metrics:')
print(classification_report(y_true, y_pred, zero_division=0))

print('\n🔍 Sample Predictions:')
for _, row in df_balanced.iterrows():
    print(f"Client {row['client_id']} | True: {row['true_label']:<13} | Pred: {row['label']:<13} | Feedback: '{row['last_feedback_text']}'")


✅ scoring_service_azure module loaded successfully.
✅ Loaded dataset:
   client_id  turnaround_time_days  error_rate_pct  communication_count              last_feedback_text
0          1                     8            12.0                    1                     Late delivery
1          2                     5             2.0                    4                     Great service
2          3                     9            10.0                    0                       No response
3          4                     4             1.0                    5                    Helpful advisor
4          5                     2             0.5                    6           Quick and accurate filing
5          6                    10            15.0                    0                    Terrible support

⚖️ Balancing dataset ...
Satisfied       3
Dissatisfied    3
Name: true_label, dtype: int64

🚀 Scoring via Azure OpenAI (or heuristic fallback)...
✅ Scoring complete:

📊 Evaluation Met